In [16]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import udf,col,max, when, desc, lit
from pyspark.sql.types import StringType, DoubleType, FloatType

import os
import h5py
import numpy as np
import pandas as pd
import tables

In [2]:
spark = SparkSession.builder \
    .appName("Test") \
    .config("spark.master", "yarn") \
    .config("spark.submit.deployMode", "client") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/08 09:00:27 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


/home/hadoop/.local/lib/python3.10/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [36]:
df = sqlContext.read.csv("hdfs://Group10-master:9000/user/hadoop/millionsongsubset.csv",
                        header='true', inferSchema='true')

df.show(vertical=True)

-RECORD 0---------------------------------------
 SongNumber              | 1                    
 SongID                  | SOMZWCG12A8C13C480   
 AlbumID                 | 300848               
 AlbumName               | Fear Itself          
 ArtistID                | ARD7TVE1187B99BFB1   
 ArtistLatitude          | null                 
 ArtistLocation          | California - LA      
 ArtistLongitude         | null                 
 ArtistName              | Casual               
 Danceability            | 0.0                  
 Duration                | 218.93179            
 KeySignature            | 1                    
 KeySignatureConfidence  | 0.736                
 Tempo                   | 92.198               
 TimeSignature           | 4                    
 TimeSignatureConfidence | 0.778                
 Title                   | I Didn't Mean To     
 Year                    | 0                    
-RECORD 1---------------------------------------
 SongNumber         

In [37]:
test = df.groupby("ArtistName").count()

test = test.orderBy(desc("count"))

test.show()



+--------------------+-----+
|          ArtistName|count|
+--------------------+-----+
|    Mario Rosenstock|   13|
|        Phil Collins|   12|
|The Jackson South...|   12|
|        Sugar Minott|   12|
|           Aerosmith|   12|
|         Mikey Dread|   11|
|Nick Cave & The B...|   11|
|         Snow Patrol|   11|
|Bill & Gloria Gai...|   11|
|         Line Renaud|   11|
|           Radiohead|   11|
|The Del McCoury Band|   10|
|             Shakira|   10|
|       Jacques Douai|   10|
|                 Ayo|   10|
|Red Hot Chili Pep...|   10|
|         White Heart|   10|
|     Michael Jackson|   10|
|              Selena|   10|
|             RUN-DMC|   10|
+--------------------+-----+
only showing top 20 rows



In [48]:
test1 = df.select(max(df["Duration"]))
val = test1.collect()[0][0]

test1 = df.filter(df["Duration"] == val)

test1.show(vertical =True)

-RECORD 0---------------------------------------
 SongNumber              | 9284                 
 SongID                  | SOKKDSX12AB018866A   
 AlbumID                 | 666530               
 AlbumName               | "Parker's Birthda... 
 ArtistID                | AROATPL1187FB48E12   
 ArtistLatitude          | 34.16788             
 ArtistLocation          | Arizona              
 ArtistLongitude         | -111.9307            
 ArtistName              | UFO                  
 Danceability            | 0.0                  
 Duration                | 1819.76771           
 KeySignature            | 2                    
 KeySignatureConfidence  | 0.601                
 Tempo                   | 134.531              
 TimeSignature           | 4                    
 TimeSignatureConfidence | 0.493                
 Title                   | Electric Phase / ... 
 Year                    | 0                    



In [60]:
test2 = df.orderBy(desc(df["Duration"]))

test2 = test2.select(df["Title"],df["Duration"])

test2.show()




+--------------------+----------+
|               Title|  Duration|
+--------------------+----------+
|Electric Phase / ...|1819.76771|
|       Endless Light| 1815.2224|
|Bitesize - Electr...| 1686.7522|
|Fall River Legend...|1610.00444|
|Live At Brixton A...| 1598.1971|
|Raga Anandi Kalya...|1519.28118|
|             Genuine|1400.37179|
|     Nomades Monades|1394.75546|
|Heaven And Hell (...|1280.88771|
|Parabolics Studies 8|  1280.522|
|Parabolics Studies 6|1266.41587|
|             911-544|1253.92934|
|Subliminal mit Na...|1222.42567|
|For Abai and Togshan|1204.29669|
|Subliminal ohne N...|1203.53914|
|Variations Sur Un...|1143.03955|
|Happiness Stan - ...|1141.08036|
|Racine dix-neuviè...|1130.94485|
|  I'm Happy She Said|  1071.882|
|Nevermind_ It's A...|1061.79873|
+--------------------+----------+
only showing top 20 rows



In [61]:
sc.stop()